In [13]:
import pandas as pd
import duckdb
import numpy as np

# Attention: count(*) et count(col) n'ont pas le même comportement

In [14]:
df = pd.DataFrame([[1,1,1,2,2,3,3],
                  [1,np.nan,np.nan,2,2,np.nan,3]]
).T
df.columns = ["col1", "col2"]

In [15]:
df

,col1,col2
0,1.0,1.0
1,1.0,NaN
2,1.0,NaN
3,2.0,2.0
4,2.0,2.0
5,3.0,NaN
6,3.0,3.0


Count(*) comptabilise les Nulls !
exemple:

In [16]:
query = """
SELECT "col1", COUNT(*)
FROM df
GROUP BY "col1"
"""
duckdb.sql(query)

┌────────┬──────────────┐
│  col1  │ count_star() │
│ double │    int64     │
├────────┼──────────────┤
│    2.0 │            2 │
│    3.0 │            2 │
│    1.0 │            3 │
└────────┴──────────────┘

Si on réalise le COUNT() sur la colonne2, les résultats sont différents !

In [17]:
query = """
SELECT "col1", COUNT(col2)
FROM df
GROUP BY "col1"
"""
duckdb.sql(query)

┌────────┬─────────────┐
│  col1  │ count(col2) │
│ double │    int64    │
├────────┼─────────────┤
│    2.0 │           2 │
│    3.0 │           1 │
│    1.0 │           1 │
└────────┴─────────────┘

In [18]:
# Les deux valeurs nulles dans la colonne 2 pour la catégorie "1" ne sont pas prises en compte
df.head(3)

,col1,col2
0,1.0,1.0
1,1.0,NaN
2,1.0,NaN


## Exemple avec les appartements

In [19]:
appt_nord = pd.read_csv("data/appartements_nord_pdc.csv")

In [20]:
query = """
SELECT Commune, COUNT(*)
FROM appt_nord
GROUP BY Commune
"""
duckdb.sql(query).df().head()

,Commune,count_star()
0,AVESNELLES,7
1,DOMPIERRE-SUR-HELPE,7
2,MORTAGNE-DU-NORD,6
3,BILLY MONTIGNY,18
4,SALLAUMINES,29


4513 ventes à Lille

In [21]:
# on insère volontairement un NULL 
appt_nord.loc[2, "valeur_fonciere"] = np.nan
appt_nord.head(3)

,Date mutation,valeur_fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Surface Carrez du 1er lot,Nombre pieces principales,Surface terrain
0,04/01/2022,208000.0,64.0,RUE,0010,DE L ABBE BONPAIN,59700.0,MARCQ EN BAROEUL,59,378,"57,69",3.0,NaN
1,04/01/2022,113000.0,1.0,RUE,0260,GUY MOCQUET,59113.0,SECLIN,59,560,"45,72",3.0,NaN
2,03/01/2022,NaN,22.0,RUE,8749,DE TREVISE,59000.0,LILLE,59,350,"62,83",3.0,NaN


In [22]:
# Le count n'a pas changé
query = """
SELECT Commune, COUNT(*)
FROM appt_nord
GROUP BY Commune
"""
duckdb.sql(query).df().head()

,Commune,count_star()
0,SAINS-DU-NORD,18
1,WIGNEHIES,12
2,LEVAL,1
3,AVESNES-LES-AUBERT,3
4,ONNAING,3


En faisant le count sur une colonne particulière, les NULLS sont exclus du count ;)

In [23]:
query = """
SELECT Commune, COUNT(valeur_fonciere)
FROM appt_nord
GROUP BY Commune
"""
duckdb.sql(query).df().head()

,Commune,count(valeur_fonciere)
0,HAUTMONT,40
1,SAINT-HILAIRE-LEZ-CAMBRAI,2
2,MARLY,59
3,ESCAUDAIN,25
4,WASNES-AU-BAC,1
